In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


geojson_map=[]
floor_image=[]
floor_info=[]

train=[]
test=[]
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
         
        if "metadata" in os.path.join(dirname, filename):
            if os.path.join(dirname, filename).endswith("geojson_map.json"):
                geojson_map.append(os.path.join(dirname, filename))
            elif os.path.join(dirname, filename).endswith("floor_image.png"):
                floor_image.append(os.path.join(dirname, filename))
            elif os.path.join(dirname, filename).endswith("floor_info.json"):
                floor_info.append(os.path.join(dirname, filename))
        elif "train" in os.path.join(dirname, filename):
            if os.path.join(dirname, filename).endswith(".txt"):
                train.append(os.path.join(dirname, filename))
        elif "test" in os.path.join(dirname, filename):
            test.append(os.path.join(dirname, filename))
        else:
            print(os.path.join(dirname, filename))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt 
  
#Metadata part can be looked later on.
img = mpimg.imread('/kaggle/input/indoor-location-navigation/metadata/5cd56c0ce2acfd2d33b6ab27/B1/floor_image.png')  
GEOpre=pd.read_json('/kaggle/input/indoor-location-navigation/metadata/5cd56c0ce2acfd2d33b6ab27/B1/geojson_map.json')
geo=pd.json_normalize(GEOpre["features"])
floor=pd.read_json("/kaggle/input/indoor-location-navigation/metadata/5cd56c0ce2acfd2d33b6ab27/B1/floor_info.json")

In [ ]:
import re

Floors = {"B2": -2, "B1": -1, "F1": 0, "F2": 1, "F3": 2, "F4": 3, "F5": 4, "F6": 5, "F7": 6, "F8": 7, "F9": 8,"1F": 0, "2F": 1, "3F": 2, "4F": 3, "5F": 4, "6F": 5, "7F": 6, "8F": 7, "9F": 8}

pd.options.mode.chained_assignment = None 
def repair(path):
    file1 = open(path, 'r')
    Lines = file1.readlines()
    updated=[]
    for line in Lines:
        if 2<=line.strip().count("TYPE_"):
            ID=[]
            for a in re.finditer(r"(?=15.{12,}TYPE_)",line):
                ID.append(a.start(0))  
            for sub in range(len(ID)):
                if sub!=len(ID)-1:
                    updated.append(line[ID[sub]:ID[sub+1]]+"\n")
                else:
                    updated.append(line[ID[sub]:])
        else:
            updated.append(line)
    with open('/kaggle/working/5cf765ece23e6e000833f03b.txt', 'w') as f:
        for item in updated:
            f.write("%s" % item)
    names = ['Time', 'Type'] + ['Feature_'+str(x) for x in range(1,9)]
    df = pd.read_csv('/kaggle/working/5cf765ece23e6e000833f03b.txt', sep='\t', comment='#', header=None, names=names, low_memory=False)
    return df

def genereateXY(row,row2):

    Time=row[0]
    X=row[1]
    Y=row[2]
    Time2=row2[0]
    X2=row2[1]
    Y2=row2[2]

    t=Time2-Time
    X=float(X)
    X2=float(X2)
    Y=float(Y)
    Y2=float(Y2)
    All=[]
    for c in range(t):

        VX=(X2-X)/t
        VY=(Y2-Y)/t

        NX=VX*c+X
        NY=VY*c+Y

        All.append([Time+c,NX,NY])
    return All


def train_DT(path_trace):
    try:   
        names = ['Time', 'Type'] + ['Feature_'+str(x) for x in range(1,9)]
        df = pd.read_csv(path_trace, sep='\t', comment='#', header=None, names=names, low_memory=False)
        Floor=Floors[path_trace.split("/")[-2]]
        ListXYZ=[]
        ListXYZXYZ=[]
        for a in df["Type"].unique():
            if a in ["TYPE_ACCELEROMETER","TYPE_MAGNETIC_FIELD","TYPE_GYROSCOPE","TYPE_ROTATION_VECTOR"]:
                a=df[df["Type"]==str(a)][["Time","Feature_1","Feature_2","Feature_3"]].rename(columns={"Feature_1":str(a)+"_X","Feature_2":str(a)+"_Y","Feature_3":str(a)+"_Z"})
                ListXYZ.append(a)
            elif  a in ["TYPE_GYROSCOPE_UNCALIBRATED","TYPE_MAGNETIC_FIELD_UNCALIBRATED","TYPE_ACCELEROMETER_UNCALIBRATED"]:
                a=df[df["Type"]==str(a)][["Time","Feature_1","Feature_2","Feature_3","Feature_4","Feature_5","Feature_6"]].rename(columns={"Feature_1":str(a)+"_X","Feature_2":str(a)+"_Y","Feature_3":str(a)+"_Z","Feature_4":str(a)+"_X2","Feature_5":str(a)+"_Y2","Feature_6":str(a)+"_Z2"})

                ListXYZXYZ.append(a)

        TYPE_WAYPOINT=df[df["Type"]=="TYPE_WAYPOINT"][["Time","Feature_1","Feature_2"]].rename(columns={"Feature_1":"X","Feature_2":"Y"})        
        TYPE_ACCELEROMETER=ListXYZ[0]
        TYPE_MAGNETIC_FIELD=ListXYZ[1]
        TYPE_GYROSCOPE=ListXYZ[2]
        TYPE_ROTATION_VECTOR=ListXYZ[3]
        TYPE_MAGNETIC_FIELD_UNCALIBRATED =ListXYZXYZ[0]
        TYPE_GYROSCOPE_UNCALIBRATED=ListXYZXYZ[1]
        TYPE_ACCELEROMETER_UNCALIBRATED=ListXYZXYZ[2]
        TYPE_WIFI=df[df["Type"]=="TYPE_WIFI"][["Time","Feature_1","Feature_2","Feature_3","Feature_4","Feature_5"]].rename(columns={"Feature_1":"ssid","Feature_2":"bssid","Feature_3":"RSSI","Feature_4":"frequency","Feature_5":"last seen timestamp"})
        TYPE_BEACON=df[df["Type"]=="TYPE_BEACON"][["Time","Feature_1","Feature_2","Feature_3","Feature_4","Feature_5","Feature_6","Feature_7","Feature_8"]].rename(columns={"Feature_1":"UUID","Feature_2":"MajorID","Feature_3":"MinorID","Feature_4":"Tx Power","Feature_5":"RSSI","Feature_6":"Distance","Feature_7":"MAC Address","Feature_8":"padding data"})



        AXX=TYPE_ROTATION_VECTOR.merge(TYPE_GYROSCOPE.merge(TYPE_ACCELEROMETER.merge(TYPE_MAGNETIC_FIELD,on='Time'),on='Time'),on='Time')
        expected=pd.DataFrame(genereateXY(TYPE_WAYPOINT.iloc[0].values,TYPE_WAYPOINT.iloc[1].values),columns=["Time","X_expected","Y_expected"])
        ALL=AXX.merge(expected,how="inner",on="Time")
        Inp=ALL[["TYPE_ROTATION_VECTOR_X","TYPE_ROTATION_VECTOR_Y","TYPE_ROTATION_VECTOR_Z","TYPE_GYROSCOPE_X","TYPE_GYROSCOPE_Y","TYPE_GYROSCOPE_Z","TYPE_ACCELEROMETER_X","TYPE_ACCELEROMETER_Y","TYPE_ACCELEROMETER_Z","TYPE_MAGNETIC_FIELD_X","TYPE_MAGNETIC_FIELD_Y","TYPE_MAGNETIC_FIELD_Z"]]
        Output=ALL[["X_expected","Y_expected"]]
        Output["Floor"]=np.ones_like(Output["X_expected"])*Floor
        return Inp,Output
    except:
        df=repair(path_trace)
        
        Floor=Floors[path_trace.split("/")[-2]]


        ListXYZ=[]
        ListXYZXYZ=[]
        for a in df["Type"].unique():
            if a in ["TYPE_ACCELEROMETER","TYPE_MAGNETIC_FIELD","TYPE_GYROSCOPE","TYPE_ROTATION_VECTOR"]:
                a=df[df["Type"]==str(a)][["Time","Feature_1","Feature_2","Feature_3"]].rename(columns={"Feature_1":str(a)+"_X","Feature_2":str(a)+"_Y","Feature_3":str(a)+"_Z"})
                ListXYZ.append(a)
            elif  a in ["TYPE_GYROSCOPE_UNCALIBRATED","TYPE_MAGNETIC_FIELD_UNCALIBRATED","TYPE_ACCELEROMETER_UNCALIBRATED"]:
                a=df[df["Type"]==str(a)][["Time","Feature_1","Feature_2","Feature_3","Feature_4","Feature_5","Feature_6"]].rename(columns={"Feature_1":str(a)+"_X","Feature_2":str(a)+"_Y","Feature_3":str(a)+"_Z","Feature_4":str(a)+"_X2","Feature_5":str(a)+"_Y2","Feature_6":str(a)+"_Z2"})

                ListXYZXYZ.append(a)

        TYPE_WAYPOINT=df[df["Type"]=="TYPE_WAYPOINT"][["Time","Feature_1","Feature_2"]].rename(columns={"Feature_1":"X","Feature_2":"Y"})        
        TYPE_ACCELEROMETER=ListXYZ[0]
        TYPE_MAGNETIC_FIELD=ListXYZ[1]
        TYPE_GYROSCOPE=ListXYZ[2]
        TYPE_ROTATION_VECTOR=ListXYZ[3]
        TYPE_MAGNETIC_FIELD_UNCALIBRATED =ListXYZXYZ[0]
        TYPE_GYROSCOPE_UNCALIBRATED=ListXYZXYZ[1]
        TYPE_ACCELEROMETER_UNCALIBRATED=ListXYZXYZ[2]
        TYPE_WIFI=df[df["Type"]=="TYPE_WIFI"][["Time","Feature_1","Feature_2","Feature_3","Feature_4","Feature_5"]].rename(columns={"Feature_1":"ssid","Feature_2":"bssid","Feature_3":"RSSI","Feature_4":"frequency","Feature_5":"last seen timestamp"})
        TYPE_BEACON=df[df["Type"]=="TYPE_BEACON"][["Time","Feature_1","Feature_2","Feature_3","Feature_4","Feature_5","Feature_6","Feature_7","Feature_8"]].rename(columns={"Feature_1":"UUID","Feature_2":"MajorID","Feature_3":"MinorID","Feature_4":"Tx Power","Feature_5":"RSSI","Feature_6":"Distance","Feature_7":"MAC Address","Feature_8":"padding data"})



        AXX=TYPE_ROTATION_VECTOR.merge(TYPE_GYROSCOPE.merge(TYPE_ACCELEROMETER.merge(TYPE_MAGNETIC_FIELD,on='Time'),on='Time'),on='Time')
        expected=pd.DataFrame(genereateXY(TYPE_WAYPOINT.iloc[0].values,TYPE_WAYPOINT.iloc[1].values),columns=["Time","X_expected","Y_expected"])
        ALL=AXX.merge(expected,how="inner",on="Time")
        Inp=ALL[["TYPE_ROTATION_VECTOR_X","TYPE_ROTATION_VECTOR_Y","TYPE_ROTATION_VECTOR_Z","TYPE_GYROSCOPE_X","TYPE_GYROSCOPE_Y","TYPE_GYROSCOPE_Z","TYPE_ACCELEROMETER_X","TYPE_ACCELEROMETER_Y","TYPE_ACCELEROMETER_Z","TYPE_MAGNETIC_FIELD_X","TYPE_MAGNETIC_FIELD_Y","TYPE_MAGNETIC_FIELD_Z"]]
        Output=ALL[["X_expected","Y_expected"]]
        Output["Floor"]=np.ones_like(Output["X_expected"])*Floor
        return Inp,Output

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
pd.options.mode.chained_assignment = None
clf = GradientBoostingRegressor(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0)
wrapper = MultiOutputRegressor(clf)
def counting():
    count=0
    for a in train[0:50]:
        Inp,Output=train_DT(a)
        wrapper.fit(Inp,Output)
        count=count+1
    return count,len(train[:50])
counting()


################# Hasn^'t finished yet ##########

In [ ]:
# file1 = open('/kaggle/input/indoor-location-navigation/train/5cd56ba1e2acfd2d33b603af/F1/5cf765ece23e6e000833f03b.txt', 'r')# A sample path
# Lines = file1.readlines()
# for line in Lines:
#     if 2==line.strip().count("TYPE_"):
#         print("orginal:",line)
#         ID=[]
#         for a in re.finditer(r"(?=15............TYPE_)",line):
#             ID.append(a.start(0))
#         print("ordered---------*****")
#         print("First:",line[0:ID[1]])
#         print("Second:",line[ID[1]:])
#         print("\n")